In [1]:
import torch
import numpy as np
from torch.autograd import Variable
import time


#Call model of layers and its forward step
from Forward_with_Layer_Setting import Net

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Call training functions of Loss functions
from NSpde_loss import lossNSpde
from MvBdry_Coefficient_and_Loss import lossMvBdry 
from BoundaryLoss import lossBdry
from InitialConditionLoss import lossIC, lossIC_with



def create_network(IC_Only_Train):
    
    net = Net()
    net = net.to(device)
    
    #Set final times for running training
    time_slices = np.array([.1,.2,.4]) #,.4,.6,.8,1
    
    #Load Training Points
    x_domain, y_domain, t_zero, x_Bdry, y_Bdry, x_l_Bdry, x_u_Bdry, y_l_Bdry, y_u_Bdry = twoDimTrainPts(net, Domain_collocation = int(1000), Bdry_collocation = int(100))
    
    start = time.time()

    #Start Training only on IC
    if IC_Only_Train == True:
        print('Training Only on the Initial Condition')
        Create_IC_Parameters(x_domain, y_domain, t_zero, 20000, 10**-3, 'IC_Only.pt', record_loss = 100, print_loss = 1000)
        IC_Done = time.time()
        print('IC Time:\t', IC_Done-start)
        
        return 0
        
    time_vec = [0, 0, 0, 0]
    
    
    #attempt to load IC if it exists
    try:
        net.load_state_dict(torch.load("IC_Only.pt"))
    except:
        pass
    
    global epsilon #used to track loss
    epsilon = []
    
    print('Training PDE')
    
    for i in range(4): #4
        #Set loop to optimize in progressively smaller learning rates
        if i == 0:
            #First loop uses progressively increasing time intervals
            print('Executing Pass 1')
            iterations = 50000
            learning_rate = 5*10**-3
        elif i == 1:
            print('Executing Pass 2')
            #time_slices = time_slices[-1]
            iterations = 20000
            learning_rate = 5*10**-4
        elif i == 2:
            print('Executing Pass 3')
            iterations = 20000
            learning_rate = 5*10**-5
        elif i ==3:
            print('Executing Pass 4')
            iterations = 2 #0000
            learning_rate = 10**-5
        
        training_loop(net, x_domain, y_domain, t_zero, x_Bdry, y_Bdry, x_l_Bdry, x_u_Bdry, 
                      y_l_Bdry, y_u_Bdry, time_slices, iterations, learning_rate, IC_coefficient = 1, record_loss = 100, print_loss = 200)
        torch.save(net.state_dict(), f"NNlayers_Bubble_{i}.pt")
        np.savetxt('epsilon.txt', epsilon)
        time_vec[i] = time.time()

    np.savetxt('epsilon.txt', epsilon)
    
    end = time.time()

    print("Total Time:\t", end-start, '\nPass 1 Time:\t', time_vec[0]-start, '\nPass 2 Time:\t', time_vec[1]-start, '\nPass 3 Time:\t', time_vec[2]-start, '\nPass 4 Time:\t', time_vec[3]-start)


def twoDimTrainPts(net, Domain_collocation, Bdry_collocation):
    #Set of all the recorded xy variables as base data for chasing during training
    
    # Domain boundary in the range [0, 1]x[0, 2] and time in [0, 1].
    x_l = net.x1_l
    x_u = net.x1_u
    y_l = net.x2_l
    y_u = net.x2_u

    #time starts at lower bound 0, ends at upper bouund updated in slices
    t_l = 0

    #Pick IC/Mv Bdry/NSpde Condition Training Random Points in Numpy
    x_domain = np.random.uniform(low= x_l, high=x_u, size=(Domain_collocation, 1)) 
    y_domain = np.random.uniform(low= y_l, high=y_u, size=(Domain_collocation, 1)) 
    
    #Move to pytorch tensors
    x_domain = Variable(torch.from_numpy(x_domain).float(), requires_grad=True).to(device)
    y_domain = Variable(torch.from_numpy(y_domain).float(), requires_grad=True).to(device)
    
    #Pick IC Training t starting points to make tensor
    t_zero = Variable(torch.zeros_like(x_domain), requires_grad=True).to(device)

    #Pick BC Training Random Points in Numpy
    x_Bdry = np.random.uniform(low=x_l, high=x_u, size=(Bdry_collocation,1))
    y_Bdry = np.random.uniform(low=y_l, high=y_u, size=(Bdry_collocation,1))       
    
    #Move to pytorch tensors
    x_Bdry= Variable(torch.from_numpy(x_Bdry).float(), requires_grad=True).to(device)
    y_Bdry = Variable(torch.from_numpy(y_Bdry).float(), requires_grad=True).to(device)
    
    ##Pick pts to make tensor for No-Slip Boundary Condition
    x_l_Bdry = Variable(x_l * torch.ones_like(x_Bdry), requires_grad=True).to(device)
    x_u_Bdry = Variable(x_u * torch.ones_like(x_Bdry), requires_grad=True).to(device)
    y_l_Bdry = Variable(y_l * torch.ones_like(x_Bdry), requires_grad=True).to(device)
    y_u_Bdry = Variable(y_u * torch.ones_like(x_Bdry), requires_grad=True).to(device)
    
            
    return x_domain, y_domain, t_zero, x_Bdry, y_Bdry, x_l_Bdry, x_u_Bdry, y_l_Bdry, y_u_Bdry
    
def tsliceTrainPts(net, Domain_collocation, Bdry_collocation, final_time):
    #Set of all the recorded t variable as base data for chasing during training

    #time starts at lower bound 0, ends at upper bouund updated in slices
    t_l = net.t_l

    #Pick IC/Mv Bdry/NSpde Condition Training Random Points in Numpy
    t_domain = np.random.uniform(low=t_l, high=final_time, size=(Domain_collocation, 1))
    
    #Move to pytorch tensors
    t_domain = Variable(torch.from_numpy(t_domain).float(), requires_grad=True).to(device)

    #Pick IC Training t starting points to make tensor
    t_zero = Variable(torch.zeros_like(t_domain), requires_grad=True).to(device)

    #Pick BC Training Random Points in Numpy
    t_Bdry = np.random.uniform(low=t_l, high=final_time, size=(Bdry_collocation,1))
    
    #Move to pytorch tensors
    t_Bdry = Variable(torch.from_numpy(t_Bdry).float(), requires_grad=True).to(device)
        
    return t_domain, t_Bdry
    
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def Create_IC_Parameters(x_domain, y_domain, t_zero, iterations, learning_rate, filename, record_loss, print_loss):
    ICnet = Net().to(device)
    
    IC_Only_training(ICnet, x_domain, y_domain, t_zero, iterations, learning_rate, record_loss, print_loss)
    
    torch.save(ICnet.state_dict(), filename)
    

def IC_Only_training(net, x_domain, y_domain, t_zero, iterations, learning_rate, record_loss, print_loss):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Domain boundary in the range [0, 1]x[0, 2] and time in [0, 1].
    #x_l = net.x1_l
    #x_u = net.x1_u
    #y_l = net.x2_l
    #y_u = net.x2_u

    ##Define Colloacation Points with Initial Condition
    #IC_collocation = collocation
    
    #define in numpy
    #x_IC = np.random.uniform(low=x_l, high=x_u, size=(IC_collocation,1))
    #y_IC = np.random.uniform(low=y_l, high=y_u, size=(IC_collocation,1))
    
    #move to pytorch tensors
    #input_x_IC = Variable(torch.from_numpy(x_IC).float(), requires_grad=True).to(device)
    #input_y_IC = Variable(torch.from_numpy(y_IC).float(), requires_grad=True).to(device)

    
    #learning rate update
    for g in net.optimizer.param_groups:
        g['lr'] = learning_rate
    
    #training loop
    epsilon_IC = [] #placeholder to track decreasing loss
    for epoch in range(1, iterations+1):
        
    
        # Resetting gradients to zero
        net.optimizer.zero_grad()
           
        #Loss based on Initial Condition
        loss = lossIC(net, x_domain, y_domain, t_zero)
           
        loss.backward()
        
        # Gradient Norm Clipping
        torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm= 5*10**2, norm_type=2, error_if_nonfinite=True)

        #Gradient Value Clipping
        #nn.utils.clip_grad_value_(net.parameters(), clip_value=1.0)
        
        net.optimizer.step()
           
        #Print Loss every 1000 Epochs
        with torch.autograd.no_grad():
            
            if epoch%record_loss == 0:
                epsilon_IC = np.append(epsilon_IC, loss.cpu().detach().numpy())
            if epoch%print_loss == 0:
                print("Iteration:", epoch, "Initial Condition Loss:", loss.data)
    
    np.savetxt('epsilon_IC.txt', epsilon_IC)

                    

def training_loop(net, x_domain, y_domain, t_zero, x_Bdry, y_Bdry, x_l_Bdry, x_u_Bdry, y_l_Bdry, y_u_Bdry, time_slices, iterations, learning_rate, IC_coefficient, record_loss, print_loss):
    global epsilon
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    #learning rate update
    for g in net.optimizer.param_groups:
        g['lr'] = learning_rate
    
    for final_time in time_slices:
        
        with torch.autograd.no_grad():
            print("Current Final Time:", final_time, "Current Learning Rate: ", get_lr(net.optimizer))  
        
        indicator = False
        reset_regularization = 1000
        
        #Iterate over these points
        
        t_domain, t_Bdry = tsliceTrainPts(net, Domain_collocation = int(1000), Bdry_collocation = int(100), final_time = final_time)    
        for epoch in range(1, iterations+1):
            # Loss calculation based on partial differential equation (PDE) 
            
            if epoch%reset_regularization == 0:
                indicator = False
    
            if epoch%reset_regularization != 0: #To detect error on forward/Backward, add hashtag on this whole line, and
            #with torch.autograd.detect_anomaly(): #use this line alternatively by deleting hashtag.
                
                ###Training steps
                # Resetting gradients to zero
                net.optimizer.zero_grad()
            
                #Loss based on Initial Condition
                mse_IC = lossIC(net, x_domain, y_domain, t_zero)
                # Gradient Norm Clipping
                torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm= 5*10**2, norm_type=1, error_if_nonfinite=False)

                #Loss based on Boundary Condition (Containing No-Slip and Free-slip)
                mse_BC = lossBdry(net, x_Bdry, y_Bdry, t_Bdry, x_l_Bdry, x_u_Bdry, y_l_Bdry, y_u_Bdry)
                # Gradient Norm Clipping
                torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm= 5*10**2, norm_type=1, error_if_nonfinite=False)

                #Loss based on PDE
                mse_NS, mse_NSdiv = lossNSpde(net, x_domain, y_domain, t_domain)
                # Gradient Norm Clipping
                torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm= 5*10**2, norm_type=1, error_if_nonfinite=False)
            
                #Loss based on Moving Boundary
                Jump_loss, SurfaceTension_loss = lossMvBdry(net, x_domain, y_domain, t_domain)
                mse_MvBdry = Jump_loss + SurfaceTension_loss
                # Gradient Norm Clipping
                torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm= 5*10**2, norm_type=1, error_if_nonfinite=False)

            
                if indicator == False:
                    indicator = True
                    IC_regular = mse_IC.detach()
                    BC_regular = mse_BC.detach()
                    pde_regular = mse_NS.detach()
                    pdediv_regular = mse_NSdiv.detach()
                    MvBdry_regular = mse_MvBdry.detach()
                
                raw_loss = IC_coefficient * mse_IC + mse_BC + mse_NS + mse_MvBdry
            
                mse_IC = mse_IC #/IC_regular
                mse_BC = mse_BC #/BC_regular
                mse_NS = mse_NS #/pde_regular
                mse_NSdiv = mse_NSdiv #/pde_regular
                mse_MvBdry = mse_MvBdry #/MvBdry_regular
            
                #Combine all Loss functions
                loss = mse_BC + 50 * mse_MvBdry + (10)**(8) *mse_IC+ mse_NS + mse_NSdiv #IC_coefficient *
                # Gradient Norm Clipping
                torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm= 5*10**2, norm_type=1, error_if_nonfinite=False)

                loss.backward()
                # Gradient Norm Clipping
                torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm= 5*10**2, norm_type=1, error_if_nonfinite=False)

            #Gradient Value Clipping
            #nn.utils.clip_grad_value_(net.parameters(), clip_value=1.0)
            net.optimizer.step()
            # Gradient Norm Clipping
            torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm= 5*10**2, norm_type=1, error_if_nonfinite=False)

            #Print Loss every 1000 Epochs
            with torch.autograd.no_grad():
                if epoch%record_loss == 0:
                    epsilon = np.append(epsilon, raw_loss.cpu().detach().numpy())
                if epoch%print_loss == 0:
                    print("Iteration:", epoch, "\tTotal Loss:", loss.data)
                    print("IC Loss: ", mse_IC.data, "\tBC Loss: ", mse_BC.data, "\tNS PDE Loss: ", mse_NS.data, "\tNS Div Free Loss: ", mse_NSdiv.data, "\tMv Bdry Jump_loss: ", Jump_loss.data ,"\tSurfaceTension_loss: ", SurfaceTension_loss.data)

            
                
create_network(True)
create_network(False)

Training Only on the Initial Condition
Iteration: 1000 Initial Condition Loss: tensor(0.0702)
Iteration: 2000 Initial Condition Loss: tensor(0.0222)
Iteration: 3000 Initial Condition Loss: tensor(0.0081)
Iteration: 4000 Initial Condition Loss: tensor(0.0039)
Iteration: 5000 Initial Condition Loss: tensor(0.0020)
Iteration: 6000 Initial Condition Loss: tensor(0.0010)
Iteration: 7000 Initial Condition Loss: tensor(0.0005)
Iteration: 8000 Initial Condition Loss: tensor(0.0003)
Iteration: 9000 Initial Condition Loss: tensor(0.0002)
Iteration: 10000 Initial Condition Loss: tensor(0.0001)
Iteration: 11000 Initial Condition Loss: tensor(8.4491e-05)
Iteration: 12000 Initial Condition Loss: tensor(5.0671e-05)
Iteration: 13000 Initial Condition Loss: tensor(2.9168e-05)
Iteration: 14000 Initial Condition Loss: tensor(1.6614e-05)
Iteration: 15000 Initial Condition Loss: tensor(9.4455e-06)
Iteration: 16000 Initial Condition Loss: tensor(5.2562e-06)
Iteration: 17000 Initial Condition Loss: tensor(4.

Iteration: 5800 	Total Loss: tensor(6796805.)
IC Loss:  tensor(0.0541) 	BC Loss:  tensor(0.0068) 	NS PDE Loss:  tensor(94.3926) 	NS Div Free Loss:  tensor(1.4822e-20) 	Mv Bdry Jump_loss:  tensor(8.6902e-07) 	SurfaceTension_loss:  tensor(27817.2109)
Iteration: 6000 	Total Loss: tensor(5357004.)
IC Loss:  tensor(0.0447) 	BC Loss:  tensor(0.0069) 	NS PDE Loss:  tensor(93.1531) 	NS Div Free Loss:  tensor(3.7028e-19) 	Mv Bdry Jump_loss:  tensor(1.5029e-06) 	SurfaceTension_loss:  tensor(17660.4707)
Iteration: 6200 	Total Loss: tensor(5689190.5000)
IC Loss:  tensor(0.0515) 	BC Loss:  tensor(0.0009) 	NS PDE Loss:  tensor(93.8731) 	NS Div Free Loss:  tensor(4.5012e-20) 	Mv Bdry Jump_loss:  tensor(3.1879e-07) 	SurfaceTension_loss:  tensor(10802.3691)
Iteration: 6400 	Total Loss: tensor(26824456.)
IC Loss:  tensor(0.2486) 	BC Loss:  tensor(0.0060) 	NS PDE Loss:  tensor(94.6730) 	NS Div Free Loss:  tensor(9.4221e-19) 	Mv Bdry Jump_loss:  tensor(8.5029e-06) 	SurfaceTension_loss:  tensor(39328.5234)

Iteration: 12400 	Total Loss: tensor(28439476.)
IC Loss:  tensor(0.2727) 	BC Loss:  tensor(5.5021e-05) 	NS PDE Loss:  tensor(95.0531) 	NS Div Free Loss:  tensor(5.1201e-18) 	Mv Bdry Jump_loss:  tensor(1.1655e-07) 	SurfaceTension_loss:  tensor(23459.6738)
Iteration: 12600 	Total Loss: tensor(4765383.)
IC Loss:  tensor(0.0436) 	BC Loss:  tensor(3.9414e-05) 	NS PDE Loss:  tensor(95.1487) 	NS Div Free Loss:  tensor(2.4551e-19) 	Mv Bdry Jump_loss:  tensor(2.4663e-08) 	SurfaceTension_loss:  tensor(8115.4453)
Iteration: 12800 	Total Loss: tensor(7372574.5000)
IC Loss:  tensor(0.0597) 	BC Loss:  tensor(0.0001) 	NS PDE Loss:  tensor(94.9547) 	NS Div Free Loss:  tensor(3.9121e-19) 	Mv Bdry Jump_loss:  tensor(1.3566e-07) 	SurfaceTension_loss:  tensor(28038.4805)
Iteration: 13000 	Total Loss: tensor(8171876.5000)
IC Loss:  tensor(0.0777) 	BC Loss:  tensor(0.0001) 	NS PDE Loss:  tensor(94.8971) 	NS Div Free Loss:  tensor(8.1671e-16) 	Mv Bdry Jump_loss:  tensor(1.0401e-07) 	SurfaceTension_loss:  ten

Iteration: 19000 	Total Loss: tensor(58722932.)
IC Loss:  tensor(0.5758) 	BC Loss:  tensor(0.0006) 	NS PDE Loss:  tensor(95.1506) 	NS Div Free Loss:  tensor(5.8588e-23) 	Mv Bdry Jump_loss:  tensor(5.2756e-09) 	SurfaceTension_loss:  tensor(22842.5625)
Iteration: 19200 	Total Loss: tensor(2792059.5000)
IC Loss:  tensor(0.0260) 	BC Loss:  tensor(2.9576e-06) 	NS PDE Loss:  tensor(95.4803) 	NS Div Free Loss:  tensor(5.2182e-24) 	Mv Bdry Jump_loss:  tensor(8.9461e-11) 	SurfaceTension_loss:  tensor(3748.9795)
Iteration: 19400 	Total Loss: tensor(48430152.)
IC Loss:  tensor(0.2303) 	BC Loss:  tensor(7.2522e-06) 	NS PDE Loss:  tensor(95.3065) 	NS Div Free Loss:  tensor(1.4605e-21) 	Mv Bdry Jump_loss:  tensor(3.8803e-09) 	SurfaceTension_loss:  tensor(507987.2812)
Iteration: 19600 	Total Loss: tensor(3506041.5000)
IC Loss:  tensor(0.0309) 	BC Loss:  tensor(1.1502e-06) 	NS PDE Loss:  tensor(95.0468) 	NS Div Free Loss:  tensor(5.1135e-22) 	Mv Bdry Jump_loss:  tensor(1.6953e-10) 	SurfaceTension_loss

Iteration: 25600 	Total Loss: tensor(67224936.)
IC Loss:  tensor(0.6540) 	BC Loss:  tensor(2.0774e-07) 	NS PDE Loss:  tensor(95.3918) 	NS Div Free Loss:  tensor(1.7316e-24) 	Mv Bdry Jump_loss:  tensor(1.1041e-11) 	SurfaceTension_loss:  tensor(36470.4180)
Iteration: 25800 	Total Loss: tensor(14645195.)
IC Loss:  tensor(0.0321) 	BC Loss:  tensor(1.7054e-07) 	NS PDE Loss:  tensor(95.4074) 	NS Div Free Loss:  tensor(1.7051e-26) 	Mv Bdry Jump_loss:  tensor(6.9259e-13) 	SurfaceTension_loss:  tensor(228739.5938)
Iteration: 26000 	Total Loss: tensor(50401076.)
IC Loss:  tensor(0.1086) 	BC Loss:  tensor(1.9377e-07) 	NS PDE Loss:  tensor(95.2758) 	NS Div Free Loss:  tensor(1.9966e-24) 	Mv Bdry Jump_loss:  tensor(8.0421e-11) 	SurfaceTension_loss:  tensor(790873.6875)
Iteration: 26200 	Total Loss: tensor(53720792.)
IC Loss:  tensor(0.5246) 	BC Loss:  tensor(1.7228e-07) 	NS PDE Loss:  tensor(95.0502) 	NS Div Free Loss:  tensor(1.1614e-25) 	Mv Bdry Jump_loss:  tensor(5.2526e-12) 	SurfaceTension_loss

...

Iteration: 11200 	Total Loss: tensor(53270.7773)
IC Loss:  tensor(3.9461e-06) 	BC Loss:  tensor(1.3046e-28) 	NS PDE Loss:  tensor(95.7818) 	NS Div Free Loss:  tensor(2.0922e-38) 	Mv Bdry Jump_loss:  tensor(0.) 	SurfaceTension_loss:  tensor(1055.6077)
Iteration: 11400 	Total Loss: tensor(13268.1465)
IC Loss:  tensor(3.9486e-05) 	BC Loss:  tensor(1.2345e-28) 	NS PDE Loss:  tensor(95.8963) 	NS Div Free Loss:  tensor(2.1158e-39) 	Mv Bdry Jump_loss:  tensor(0.) 	SurfaceTension_loss:  tensor(184.4724)
Iteration: 11600 	Total Loss: tensor(547.4464)
IC Loss:  tensor(5.5230e-07) 	BC Loss:  tensor(1.1563e-28) 	NS PDE Loss:  tensor(95.9444) 	NS Div Free Loss:  tensor(1.0315e-39) 	Mv Bdry Jump_loss:  tensor(0.) 	SurfaceTension_loss:  tensor(7.9254)
Iteration: 11800 	Total Loss: tensor(454.7750)
IC Loss:  tensor(4.6818e-07) 	BC Loss:  tensor(1.0488e-28) 	NS PDE Loss:  tensor(95.9471) 	NS Div Free Loss:  tensor(9.6312e-40) 	Mv Bdry Jump_loss:  tensor(0.) 	SurfaceTension_loss:  tensor(6.2402)
Iterati

Iteration: 18000 	Total Loss: tensor(97.6526)
IC Loss:  tensor(1.8027e-08) 	BC Loss:  tensor(1.3215e-24) 	NS PDE Loss:  tensor(95.8499) 	NS Div Free Loss:  tensor(4.7694e-38) 	Mv Bdry Jump_loss:  tensor(0.) 	SurfaceTension_loss:  tensor(6.2343e-10)
Iteration: 18200 	Total Loss: tensor(2.6768e+11)
IC Loss:  tensor(3.7164e-08) 	BC Loss:  tensor(1.8728e-31) 	NS PDE Loss:  tensor(20.4272) 	NS Div Free Loss:  tensor(1.1352e-33) 	Mv Bdry Jump_loss:  tensor(0.) 	SurfaceTension_loss:  tensor(5.3535e+09)
Iteration: 18400 	Total Loss: tensor(196.0437)
IC Loss:  tensor(1.0267e-06) 	BC Loss:  tensor(5.4863e-24) 	NS PDE Loss:  tensor(91.9571) 	NS Div Free Loss:  tensor(7.7962e-37) 	Mv Bdry Jump_loss:  tensor(0.) 	SurfaceTension_loss:  tensor(0.0284)
Iteration: 18600 	Total Loss: tensor(2.0500e+08)
IC Loss:  tensor(0.2000) 	BC Loss:  tensor(4.1295e-30) 	NS PDE Loss:  tensor(84.0530) 	NS Div Free Loss:  tensor(3.7372e-35) 	Mv Bdry Jump_loss:  tensor(0.) 	SurfaceTension_loss:  tensor(3699981.5000)
Ite

Iteration: 24800 	Total Loss: tensor(184.4412)
IC Loss:  tensor(1.8443e-06) 	BC Loss:  tensor(7.3748e-06) 	NS PDE Loss:  tensor(0.0117) 	NS Div Free Loss:  tensor(1.5963e-32) 	Mv Bdry Jump_loss:  tensor(0.) 	SurfaceTension_loss:  tensor(0.)
Iteration: 25000 	Total Loss: tensor(348.4966)
IC Loss:  tensor(3.4850e-06) 	BC Loss:  tensor(1.3932e-05) 	NS PDE Loss:  tensor(4.1835e-06) 	NS Div Free Loss:  tensor(2.1352e-37) 	Mv Bdry Jump_loss:  tensor(0.) 	SurfaceTension_loss:  tensor(0.)
Iteration: 25200 	Total Loss: tensor(7.2318)
IC Loss:  tensor(4.6294e-08) 	BC Loss:  tensor(1.7632e-07) 	NS PDE Loss:  tensor(1.1341) 	NS Div Free Loss:  tensor(2.8230e-33) 	Mv Bdry Jump_loss:  tensor(0.) 	SurfaceTension_loss:  tensor(0.0294)
Iteration: 25400 	Total Loss: tensor(19.2048)
IC Loss:  tensor(1.9190e-07) 	BC Loss:  tensor(7.6221e-07) 	NS PDE Loss:  tensor(0.0151) 	NS Div Free Loss:  tensor(2.2547e-33) 	Mv Bdry Jump_loss:  tensor(0.) 	SurfaceTension_loss:  tensor(0.)
Iteration: 25600 	Total Loss: t

Iteration: 31600 	Total Loss: tensor(30005660.)
IC Loss:  tensor(0.3000) 	BC Loss:  tensor(8.2872e-20) 	NS PDE Loss:  tensor(95.8809) 	NS Div Free Loss:  tensor(3.0139e-38) 	Mv Bdry Jump_loss:  tensor(2.0140e-24) 	SurfaceTension_loss:  tensor(32.3998)
Iteration: 31800 	Total Loss: tensor(30000890.)
IC Loss:  tensor(0.3000) 	BC Loss:  tensor(8.2769e-20) 	NS PDE Loss:  tensor(95.8950) 	NS Div Free Loss:  tensor(2.8978e-38) 	Mv Bdry Jump_loss:  tensor(1.4648e-24) 	SurfaceTension_loss:  tensor(9.6317)
Iteration: 32000 	Total Loss: tensor(30000978.)
IC Loss:  tensor(0.3000) 	BC Loss:  tensor(8.2428e-20) 	NS PDE Loss:  tensor(95.9012) 	NS Div Free Loss:  tensor(3.3781e-38) 	Mv Bdry Jump_loss:  tensor(1.5293e-24) 	SurfaceTension_loss:  tensor(10.6048)
Iteration: 32200 	Total Loss: tensor(30001546.)
IC Loss:  tensor(0.3000) 	BC Loss:  tensor(8.3268e-20) 	NS PDE Loss:  tensor(95.8900) 	NS Div Free Loss:  tensor(2.7000e-38) 	Mv Bdry Jump_loss:  tensor(1.1924e-24) 	SurfaceTension_loss:  tensor(17

RuntimeError: all elements of input should be between 0 and 1